In [1]:
import pandas as pd

# Load raw data
df = pd.read_csv(r"C:\Users\luket\projects\Technical Support/Technical Support Dataset.csv")

# Preview
df.head()

# Overview
df.info()

# Descriptive stats
df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2330 entries, 0 to 2329
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Status                          2330 non-null   object 
 1   Ticket ID                       2330 non-null   int64  
 2   Priority                        2330 non-null   object 
 3   Source                          2330 non-null   object 
 4   Topic                           2330 non-null   object 
 5   Agent Group                     2330 non-null   object 
 6   Agent Name                      2330 non-null   object 
 7   Created time                    2330 non-null   object 
 8   Expected SLA to resolve         2330 non-null   object 
 9   Expected SLA to first response  2330 non-null   object 
 10  First response time             2312 non-null   object 
 11  SLA For first response          2330 non-null   object 
 12  Resolution time                 19

,Status,Ticket ID,Priority,Source,Topic,Agent Group,Agent Name,Created time,Expected SLA to resolve,Expected SLA to first response,...,Resolution time,SLA For Resolution,Close time,Agent interactions,Survey results,Product group,Support Level,Country,Latitude,Longitude
count,2330,2330.000000,2330,2330,2330,2330,2330,2330,2330,2330,...,1912,2330,1173,2312.000000,1173.000000,2330,2330,2330,2330.000000,2330.000000
unique,4,NaN,3,3,8,2,8,2330,2330,2330,...,1912,2,1173,NaN,NaN,4,2,12,NaN,NaN
top,Closed,NaN,Low,Email,Product setup,1st line support,Nicola Wane,2023-01-02 00:58:36,2023-01-04 00:58:36,2023-01-02 01:58:36,...,2023-01-04 00:31:51.694,Within SLA,2023-01-04 04:02:59.013,NaN,NaN,Ready to use Software,Tier 1,Germany,NaN,NaN
freq,1173,NaN,1192,1234,630,1770,367,1,1,1,...,1,1547,1,NaN,NaN,1010,1770,306,NaN,NaN
mean,NaN,2481.857082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.470156,3.510656,NaN,NaN,NaN,48.260756,9.849723
std,NaN,858.040637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.130958,1.240361,NaN,NaN,NaN,5.085409,9.814618
min,NaN,1012.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,39.368279,-8.019675
25%,NaN,1726.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.000000,3.000000,NaN,NaN,NaN,43.036776,2.349014
50%,NaN,2480.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.000000,4.000000,NaN,NaN,NaN,48.864716,12.678565
75%,NaN,3233.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.000000,4.000000,NaN,NaN,NaN,52.536273,15.474913


In [2]:
# Change key time fields to datetime

time_cols = ['Created time', 'First response time', 'Resolution time', 'Close time',
             'Expected SLA to first response', 'Expected SLA to resolve']
df[time_cols] = df[time_cols].apply(pd.to_datetime, errors='coerce')

#Check for nulls all features
df.isnull().sum()


Status                               0
Ticket ID                            0
Priority                             0
Source                               0
Topic                                0
Agent Group                          0
Agent Name                           0
Created time                         0
Expected SLA to resolve              0
Expected SLA to first response       0
First response time                 18
SLA For first response               0
Resolution time                    418
SLA For Resolution                   0
Close time                        1157
Agent interactions                  18
Survey results                    1157
Product group                        0
Support Level                        0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [3]:
# treat missing values in "First response time"
missing_first_response_tickets = df[df['First response time'].isnull()]
print(missing_first_response_tickets['Status'].value_counts())


Status
Open    18
Name: count, dtype: int64


- Since the status of these tickets are of "Open", these tickets are newly created or still in the queue, and an agent hasn't sent their first response yet. 
- For an Open ticket, not having a First response time is the expected and correct state.
- Therefore, they will remain in dataset for analyses like overall ticket volume, distribution by Source, Priority, Topic, etc., as they are valid, active tickets.

In [4]:
#  Treat missing values in "Agent interactions"

#  Given what we just found about First response time, there's a very high chance these are the exact same 18 Open tickets that haven't received a first response yet. 
#  If no response has been sent, there would naturally be zero agent interactions logged.

are_they_the_same_missing_rows = df['First response time'].isnull() == df['Agent interactions'].isnull()
print(are_they_the_same_missing_rows.all())

True


- Confirmed that the 18 missing Agent interactions values are for the same 18 Open tickets that haven't received a First response time yet
-  Therefore, 0 agent interactions is the representation for these tickets.

In [5]:
# Fill missing values in 'Agent interactions' with 0
df['Agent interactions'].fillna(0, inplace=True)

C:\Users\luket\AppData\Local\Temp\ipykernel_6104\129250371.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Agent interactions'].fillna(0, inplace=True)


In [6]:
# Treat missing values in "Resolution time"

# Missing Resolution time most likely means the ticket is still Open or In Progress

# Inspect
missing_resolution_tickets = df[df['Resolution time'].isnull()]
print(missing_resolution_tickets['Status'].value_counts())

Status
In progress    400
Open            18
Name: count, dtype: int64


- Confirmed that these ticket that are still being actively worked on or waiting to be picked up.
- Leave them as NaN
- When analyzing Resolution time or analyze SLA For Resolution compliance, Pandas will exclude these NaN values, ensuring metrics are based only on tickets that have been resolved.

In [7]:
# Inspect missing values in "Close timne" and "Survey Results"

# For missing Close time
missing_close_time_tickets = df[df['Close time'].isnull()]
print("Status for missing Close time:")
print(missing_close_time_tickets['Status'].value_counts())

# For missing Survey results
missing_survey_results_tickets = df[df['Survey results'].isnull()]
print("\nStatus for missing Survey results:")
print(missing_survey_results_tickets['Status'].value_counts())

# Check if the missing values align perfectly for both columns
are_close_and_survey_missing_same_rows = df['Close time'].isnull() == df['Survey results'].isnull()
print("\nDo missing values for Close time and Survey results align perfectly?")
print(are_close_and_survey_missing_same_rows.all())

Status for missing Close time:
Status
Resolved       739
In progress    400
Open            18
Name: count, dtype: int64

Status for missing Survey results:
Status
Resolved       739
In progress    400
Open            18
Name: count, dtype: int64

Do missing values for Close time and Survey results align perfectly?
True


- We have "In progress" (400) and "Open" (18) tickets with missing Close time and Survey results.
    - These tickets are still active and haven't reached the final stages of the workflow. Therefore, it's expected that they wouldn't have a "Close time yet", and a "Survey" results would typically only be requested after the ticket is closed 
    - These NaNs are meaningful and accurate.

- We have "Resolved" (739) tickets with missing "Close time" and "Survey" results.
    - If a ticket is marked Resolved, it should ideally have a Close time. 
    - The missing survey result implies the customer simply didn't respond. 
    - Leave missing survey result as nan, because Imputing Survey results (e.g., with the average or median) would be highly misleading
    - Further Analysis: While you don't impute, the fact that 739 resolved tickets don't have a survey response is a significan. 
        - Might explore why this is happening later (e.g., are surveys not being sent out consistently, or are customers actively choosing not to respond for certain types of issues/channels?).

In [8]:
# Cleaning strategies for "Close time" for Resolved Tickets:
#   Impute "Close time" with "Resolution time" for Resolved tickets.

# Identify rows where Status is 'Resolved' AND 'Close time' is missing
condition = (df['Status'] == 'Resolved') & (df['Close time'].isnull())

# Fill 'Close time' in these specific rows with the 'Resolution time'
df.loc[condition, 'Close time'] = df.loc[condition, 'Resolution time']

In [9]:
# Tthe current status of all missing values
print(df.isnull().sum())

Status                               0
Ticket ID                            0
Priority                             0
Source                               0
Topic                                0
Agent Group                          0
Agent Name                           0
Created time                         0
Expected SLA to resolve              0
Expected SLA to first response       0
First response time                 18
SLA For first response               0
Resolution time                    418
SLA For Resolution                   0
Close time                         418
Agent interactions                   0
Survey results                    1157
Product group                        0
Support Level                        0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64


- Done with handling missing values

In [10]:
# Handling Categorical features
# List of likely categorical columns
categorical_cols = [
    'Status', 'Priority', 'Source', 'Topic', 'Agent Group', 'Agent Name',
    'SLA For first response', 'SLA For Resolution', 'Product group',
    'Support Level', 'Country'
]

for col in categorical_cols:
    if df[col].dtype == 'object': # Ensure it's a string/object type before applying str methods
        df[col] = df[col].astype(str).str.lower().str.strip() # Cover to string and standarize
        
    print(f"\n--- Value Counts for '{col}' ---")
    print(df[col].value_counts())
    print("-" * 30)


--- Value Counts for 'Status' ---
Status
closed         1173
resolved        739
in progress     400
open             18
Name: count, dtype: int64
------------------------------

--- Value Counts for 'Priority' ---
Priority
low       1192
medium     722
high       416
Name: count, dtype: int64
------------------------------

--- Value Counts for 'Source' ---
Source
email    1234
chat      850
phone     246
Name: count, dtype: int64
------------------------------

--- Value Counts for 'Topic' ---
Topic
product setup               630
pricing and licensing       525
feature request             417
purchasing and invoicing    264
bug report                  225
other                       203
training request             66
Name: count, dtype: int64
------------------------------

--- Value Counts for 'Agent Group' ---
Agent Group
1st line support    1770
2nd line support     560
Name: count, dtype: int64
------------------------------

--- Value Counts for 'Agent Name' ---
Agent Name
ni

- Categorical columns are clean and consistent.

In [11]:
# Save clean dataset
output_file_path = 'technical_support_data_cleaned.csv'
df.to_csv(output_file_path, index=False)
print(f"Cleaned dataset saved successfully to: {output_file_path}")

Cleaned dataset saved successfully to: technical_support_data_cleaned.csv
